In [1]:
import AFQ.data as afd
import AFQ.registration as afr
data_dict = afd.fetch_hcp([994273])

/Users/tpawlowski/workspace/dokstud/mri_pipeline/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from dipy.reconst import dti
from dipy.core.gradients import gradient_table
import os.path as op
home = op.expanduser('~')
gtab = gradient_table(op.join(home, 'AFQ_data/HCP/sub-994273/sess-01/dwi/sub-994273_dwi.bval'), 
                      op.join(home, 'AFQ_data/HCP/sub-994273/sess-01/dwi/sub-994273_dwi.bvec'),
                      b0_threshold=10)

In [3]:
import nibabel as nib
img = nib.load(op.join(home, 'AFQ_data/HCP/sub-994273/sess-01/dwi/sub-994273_dwi.nii.gz'))
data = img.get_data()

In [4]:
reference_data = data[..., 0]

In [5]:
from dipy.segment.mask import median_otsu
_, mask = median_otsu(reference_data, 4, 2, False, dilate=1)

In [6]:
import numpy as np

print(mask.shape)
print(np.count_nonzero(mask))

for x in range(10):
    mask[1:, :, :] = np.logical_and(mask[1:, :, :], mask[:-1, :, :])
    mask[:-1, :, :] = np.logical_and(mask[1:, :, :], mask[:-1, :, :])
    mask[:, 1:, :] = np.logical_and(mask[:, 1:, :], mask[:, :-1, :])
    mask[:, :-1, :] = np.logical_and(mask[:, 1:, :], mask[:, :-1, :])
    
print(np.count_nonzero(mask))

(145, 174, 145)
768172
404905


In [7]:
fit_data = data[..., 0, None]

In [ ]:
big_delta = 0.0365  # seconds
small_delta = 0.0157  # seconds

for idx in np.arange(1, 288):
    print("Fit data shape {0}".format(fit_data.shape))
    
    print("Incoming data: ", idx)
    this_data = data[..., idx]

    print("Register incoming data to the reference image")
    this_data, _ = afr.affine_registration(this_data,
                                                    reference_data)
    print("Create full series for this iteration")
    fit_data = np.concatenate([fit_data, this_data[..., None]], axis=-1)
    
    if idx > 20:
        print("Create the gradient table for this iteration")
        this_gtab = gradient_table(bvals=gtab.bvals[:idx + 1], 
                                   bvecs=gtab.bvecs[:idx + 1],
                                   big_delta=big_delta,
                                   small_delta=small_delta, 
                                   b0_threshold=10)

        print("Initialize model")
        this_model = dti.TensorModel(this_gtab)

        print("Fit the model in each voxel")
        this_model.fit(fit_data, mask=mask)

Fit data shape (145, 174, 145, 1)
Incoming data:  1
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Fit data shape (145, 174, 145, 2)
Incoming data:  2
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Fit data shape (145, 174, 145, 3)
Incoming data:  3
Register incoming data to th